In [33]:
# For using custom module.
import sys
sys.path.append('../custom_library')

In [35]:
import numpy as np
import pandas as pd 
import connect_to_db as cn
import yake

In [114]:
def extract_keywords(doc):
    kw_extractor = yake.KeywordExtractor()
    language = "en"
    max_ngram_size = 1
    deduplication_threshold = 0.9
    numOfKeywords = 5
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    
    keyword_list = []
    for text in doc:
        keywords = custom_kw_extractor.extract_keywords(text)
        for kw in keywords:
            if 'â' not in kw[0]:
                keyword_list.append(kw[0].lower())
        
    return keyword_list

In [115]:
def save_keywords(table_name, author, degree):
    doc = []

    if degree == 'outdegree':
        sql = f"select distinct body from comments where author='{author}' and link_key = parent_key and is_valid_author=1 and is_valid=1;"
    else:
        sql = 'test'
    result_df = cn.select_query_result_to_df(sql)
    if not result_df.empty:
        if degree == 'outdegree':
            bodies = np.array(result_df['body'].astype(str).values.tolist())
            doc.extend(bodies)

    return extract_keywords(doc)

In [116]:
sql = "select node_id from nodes where top_k_outdegree=0.1;"
result_df = cn.select_query_result_to_df(sql)
hub_users = list(np.array(result_df['node_id'].values.tolist()))

keywords_list = []

for user in hub_users:
    keywords_list.append(save_keywords('nodes', user, 'outdegree'))

In [117]:
key_set = list(set(list(sum(keywords_list, []))))
counts = [0] * len(key_set)
print(len(counts), len(key_set))

all_keywords = list(sum(keywords_list, []))

for i in range(len(all_keywords)):
    for j in range(len(key_set)):
        if all_keywords[i] == key_set[j]:
            counts[j] += 1
            break

34080 34080


In [118]:
word_count_pair = []
for word, count in zip(key_set, counts):
    word_count_pair.append([word, count])

pair_dict = dict(word_count_pair)
sorted_pair_dict = sorted(pair_dict.items(), key = lambda item: item[1], reverse = True)

In [113]:
print(sorted_pair_dict[:50])

[('people', 745), ('feel', 728), ('good', 512), ('time', 446), ('hey', 441), ('talk', 423), ('yeah', 366), ('life', 364), ('feeling', 299), ('things', 290), ('hope', 247), ('sounds', 233), ('love', 231), ('anxiety', 230), ('happy', 218), ('day', 216), ('bpd', 211), ('lot', 210), ('work', 209), ('years', 201), ('give', 195), ('meds', 194), ('make', 193), ('man', 192), ('hard', 192), ('thing', 190), ('dont', 168), ('person', 167), ('find', 163), ('friends', 160), ('bad', 158), ('psychiatrist', 149), ('therapy', 149), ('wanna', 148), ('dude', 148), ('depression', 137), ('symptoms', 134), ('friend', 133), ('great', 132), ('therapist', 131), ('sharing', 124), ('schizophrenia', 124), ('stop', 123), ('normal', 123), ('understand', 123), ('doctor', 119), ('wanna talk', 113), ('kind', 112), ('back', 111), ('trauma', 109)]


In [120]:
print(sorted_pair_dict[:100])

[('people', 15259), ('feel', 13342), ('time', 9433), ('life', 8545), ('good', 7990), ('things', 7348), ('sounds', 5196), ('lot', 4933), ('anxiety', 4921), ('talk', 4706), ('depression', 4613), ('make', 4455), ('work', 4199), ('yeah', 4083), ('years', 4077), ('thing', 4045), ('hope', 3909), ('find', 3825), ('feeling', 3789), ('therapist', 3755), ('love', 3703), ('person', 3594), ('friends', 3562), ('therapy', 3533), ('bad', 3440), ('day', 3430), ('hard', 3272), ('hey', 3078), ('post', 2721), ('happy', 2684), ('thoughts', 2647), ('friend', 2573), ('ocd', 2504), ('back', 2501), ('family', 2468), ('trauma', 2465), ('hear', 2458), ('understand', 2365), ('care', 2338), ('give', 2322), ('long', 2220), ('great', 2205), ('kind', 2139), ('meds', 2131), ('parents', 2129), ('feelings', 2108), ('dont', 2088), ('pain', 2043), ('experience', 2016), ('doctor', 2000), ('relationship', 1970), ('normal', 1951), ('happened', 1942), ('mental', 1941), ('pretty', 1933), ('man', 1900), ('mind', 1890), ('job',

In [138]:
top_words = []
for pair in sorted_pair_dict:
    top_words.append(pair[0])

In [139]:
top_words

['people',
 'feel',
 'time',
 'life',
 'good',
 'things',
 'sounds',
 'lot',
 'anxiety',
 'talk',
 'depression',
 'make',
 'work',
 'yeah',
 'years',
 'thing',
 'hope',
 'find',
 'feeling',
 'therapist',
 'love',
 'person',
 'friends',
 'therapy',
 'bad',
 'day',
 'hard',
 'hey',
 'post',
 'happy',
 'thoughts',
 'friend',
 'ocd',
 'back',
 'family',
 'trauma',
 'hear',
 'understand',
 'care',
 'give',
 'long',
 'great',
 'kind',
 'meds',
 'parents',
 'feelings',
 'dont',
 'pain',
 'experience',
 'doctor',
 'relationship',
 'normal',
 'happened',
 'mental',
 'pretty',
 'man',
 'mind',
 'job',
 'situation',
 'school',
 'wrong',
 'makes',
 'thought',
 'start',
 'stuff',
 'symptoms',
 'bpd',
 'abuse',
 'live',
 'deal',
 'medication',
 'year',
 'relate',
 'made',
 'health',
 'read',
 'times',
 'shit',
 'mom',
 'stop',
 'world',
 'worse',
 'change',
 'problem',
 'sleep',
 'hate',
 'place',
 'point',
 'part',
 'advice',
 'ptsd',
 'lol',
 'nice',
 'glad',
 'remember',
 'real',
 'bipolar',
 'da

In [175]:
sql2 = "select node_id from nodes where top_k_outdegree=10 limit 697;"
result_df2 = cn.select_query_result_to_df(sql2)
nonhub_users = list(np.array(result_df2['node_id'].values.tolist()))

keywords_list2 = []

for user in nonhub_users:
    keywords_list2.append(save_keywords('nodes', user, 'outdegree'))

In [176]:
key_set2 = list(set(list(sum(keywords_list2, []))))
counts2 = [0] * len(key_set2)
print(len(counts2), len(key_set2))

all_keywords2 = list(sum(keywords_list2, []))

for i in range(len(all_keywords2)):
    for j in range(len(key_set2)):
        if all_keywords2[i] == key_set2[j]:
            counts2[j] += 1
            break

5876 5876


In [177]:
word_count_pair2 = []
for word, count in zip(key_set2, counts2):
    word_count_pair2.append([word, count])

pair_dict2 = dict(word_count_pair2)
sorted_pair_dict2 = sorted(pair_dict2.items(), key = lambda item: item[1], reverse = True)

In [137]:
print(sorted_pair_dict2[:100])

[('feel', 128), ('people', 80), ('life', 66), ('happy', 61), ('time', 60), ('good', 60), ('hope', 42), ('love', 41), ('talk', 37), ('things', 35), ('day', 35), ('thing', 35), ('hard', 30), ('birthday', 30), ('years', 29), ('friend', 29), ('feeling', 29), ('dont', 29), ('find', 27), ('hey', 27), ('bad', 26), ('lot', 25), ('work', 24), ('person', 24), ('dude', 24), ('year', 23), ('sounds', 23), ('make', 22), ('man', 22), ('friends', 22), ('times', 21), ('understand', 21), ('post', 20), ('shit', 20), ('great', 19), ('anxiety', 19), ('world', 19), ('thoughts', 18), ('makes', 18), ('family', 17), ('feels', 17), ('pretty', 17), ('stop', 17), ('experience', 17), ('yeah', 17), ('relate', 17), ('proud', 16), ('care', 16), ('wrong', 16), ('lol', 16), ('live', 16), ('remember', 15), ('totally', 15), ('part', 15), ('real', 15), ('job', 15), ('fucking', 14), ('hear', 13), ('situation', 13), ('ocd', 13), ('therapy', 13), ('pain', 13), ('give', 13), ('mind', 13), ('glad', 13), ('loved', 12), ('true',

In [178]:
top_words2 = []
for pair in sorted_pair_dict2:
    top_words2.append(pair[0])

In [179]:
hub_words = [item for item in top_words[:230] if item not in top_words2[:230]]
for pair in sorted_pair_dict[:230]:
    for word in hub_words:
        if pair[0] == word:
            print(pair)

('brain', 1230)
('disorder', 1191)
('child', 1147)
('depressed', 1077)
('online', 1069)
('psychiatrist', 1056)
('psychosis', 1037)
('common', 1032)
('check', 1020)
('anger', 1006)
('emotional', 1001)
('age', 954)
('control', 948)
('side', 948)
('treat', 937)
('important', 933)
('overcoming', 923)
('difficult', 918)
('learn', 911)
('diagnosis', 911)
('schizophrenia', 900)
('mother', 884)
('cptsd', 878)
('leave', 853)
('illness', 846)
('sense', 830)
('helpful', 828)
('tough', 814)
('overview', 807)
('treatment', 803)
('break', 789)
('hell', 787)
('sex', 786)
('women', 784)
('bud', 774)
('tips', 770)
('future', 768)
('kids', 768)
('issue', 756)
('heard', 732)
('safe', 726)
('answer', 724)
('worry', 701)
('stress', 699)
('college', 697)
('relationships', 692)


In [180]:
hub_words2 = [item for item in top_words2[:230] if item not in top_words[:230]]
for pair in sorted_pair_dict2[:230]:
    for word in hub_words2:
        if pair[0] == word:
            print(pair)

('proud', 79)
('literally', 47)
('fucking', 47)
('congratulations', 46)
('exact', 42)
('congrats', 41)
('beautiful', 39)
('awesome', 35)
('gonna', 34)
('cry', 31)
('brother', 31)
('loved', 31)
('bro', 31)
('worst', 31)
('music', 30)
('sending', 30)
('omg', 30)
('ðÿ’', 30)
('idk', 29)
('recently', 29)
('super', 29)
('hit', 28)
('anxious', 28)
('panic', 28)
('week', 28)
('boyfriend', 27)
('hugs', 27)
('awful', 27)
('heart', 26)
('wanna', 26)
('step', 26)
('lonely', 26)
('imagine', 26)
('weeks', 24)
('angry', 24)
('easy', 24)
('small', 24)
('left', 23)
('recommend', 23)
('telling', 23)
('enjoy', 23)
('damn', 23)
('hang', 23)
('needed', 23)
('share', 22)
('experienced', 22)
